<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
!python -m spacy validate

✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.0.6) =================
ℹ spaCy installation:
/Users/zuoyou/opt/anaconda3/lib/python3.7/site-packages/spacy

NAME              SPACY            VERSION                            
en_core_web_lg    >=3.0.0,<3.1.0   3.0.0   ✔
en_core_web_md    >=3.0.0,<3.1.0   3.0.0   ✔
en_core_web_trf   >=3.0.0,<3.1.0   3.0.0   ✔



In [3]:
import spacy 

ner = spacy.load("../03_spaCy_ner/output/G06T_2015/model-best/")
nlp = spacy.load('en_core_web_md')

s2v = nlp.add_pipe("sense2vec")
s2v.from_disk("./s2v_old/")

In [4]:
import numpy as np 

def most_similar(word, topn=5):
    queries = np.asarray([word.vector])   
    keys, best_rows, scores = nlp.vocab.vectors.most_similar(queries, n=topn)
    return [nlp.vocab.strings[key] for key in keys[0]]


def find_synonym(term, nlp, n=10): 
    term = nlp(term)[:]
    if term._.in_s2v:
        return term._.s2v_most_similar(n=n)
    else:
        return most_similar(term, topn=n)

In [16]:
find_synonym("SVM", n=10, nlp=nlp)

['Tlt',
 'GTA',
 'Igi',
 'Graw',
 'rabel',
 '64-Player',
 'INTEC',
 'thps3',
 'porcelain-enameled',
 'CREDIT-GRANTING']

In [20]:
find_synonym("patternxxx analysis", n=10, nlp=nlp)

['ANALYSES',
 'CLIMATOLOGICAL',
 'ANALYTICAL',
 'five-step',
 'EVALUATIONS',
 'Self-Evaluation',
 'DISSECT',
 'Assaying',
 'COMPARATIVE',
 'studies']

In [17]:
find_synonym("machine learning", n=10, nlp=nlp)

[(('data science', 'NOUN'), 0.9005),
 (('natural language processing', 'NOUN'), 0.8987),
 (('computer vision', 'NOUN'), 0.8978),
 (('deep learning', 'NOUN'), 0.8963),
 (('data analysis', 'NOUN'), 0.8924),
 (('Machine learning', 'NOUN'), 0.8842),
 (('neural nets', 'NOUN'), 0.8755),
 (('Machine Learning', 'ORG'), 0.8642),
 (('bioinformatics', 'NOUN'), 0.8624),
 (('neural networks', 'NOUN'), 0.858)]

In [18]:
find_synonym("computer vision", n=10, nlp=nlp)

[(('machine learning', 'NOUN'), 0.8978),
 (('deep learning', 'NOUN'), 0.8874),
 (('natural language processing', 'NOUN'), 0.8636),
 (('neural nets', 'NOUN'), 0.8481),
 (('machine vision', 'NOUN'), 0.8431),
 (('image recognition', 'NOUN'), 0.8342),
 (('Machine learning', 'NOUN'), 0.8228),
 (('Machine Learning', 'ORG'), 0.8143),
 (('signal processing', 'NOUN'), 0.8127),
 (('computer graphics', 'NOUN'), 0.8097)]

In [15]:
find_synonym("pixel", n=10, nlp=nlp)

[(('single pixel', 'NOUN'), 0.8974),
 (('4 pixels', 'QUANTITY'), 0.8569),
 (('2 pixels', 'QUANTITY'), 0.8238),
 (('resulting image', 'NOUN'), 0.8122),
 (('two pixels', 'QUANTITY'), 0.8078),
 (('pixel', 'VERB'), 0.8053),
 (('pixel resolution', 'NOUN'), 0.8026),
 (('polygons', 'NOUN'), 0.8021),
 (('individual pixels', 'NOUN'), 0.798),
 (('pixel size', 'NOUN'), 0.7972)]

In [13]:
find_synonym("method", n=10, nlp=nlp)

[(('particular method', 'NOUN'), 0.8683),
 (('different method', 'NOUN'), 0.8621),
 (('specific method', 'NOUN'), 0.8488),
 (('methods', 'NOUN'), 0.8482),
 (('other method', 'NOUN'), 0.8472),
 (('second method', 'NOUN'), 0.8419),
 (('other methods', 'NOUN'), 0.8312),
 (('multiple methods', 'NOUN'), 0.8286),
 (('first method', 'NOUN'), 0.8286),
 (('better method', 'NOUN'), 0.8154)]

In [7]:
import datetime
doc = nlp("A sentence about machine learning.")

start_time = datetime.datetime.now()
print(find_synonym(doc[3:5].text,nlp))
end_time = datetime.datetime.now()    

interval = (end_time-start_time).seconds   

[(('data science', 'NOUN'), 0.9005), (('natural language processing', 'NOUN'), 0.8987), (('computer vision', 'NOUN'), 0.8978), (('deep learning', 'NOUN'), 0.8963), (('data analysis', 'NOUN'), 0.8924), (('Machine learning', 'NOUN'), 0.8842), (('neural nets', 'NOUN'), 0.8755), (('Machine Learning', 'ORG'), 0.8642), (('bioinformatics', 'NOUN'), 0.8624), (('neural networks', 'NOUN'), 0.858)]


In [8]:
print ("It takes\t", interval, "seconds")

It takes	 4 seconds


In [9]:
with open('../data/G06T0003000000.txt', encoding = 'utf-8') as f:
    patents = f.read().split('\n\n\n')

In [11]:
from tqdm import tqdm
doc_ner = ner(patents[0])
ents = set([ent.text for ent in doc_ner.ents])

for ent in ents:
    print(find_synonym(ent, n=10, nlp=nlp))

['METHOD', 'CLAIM', 'METHODS', 'oF', 'Of', 'of', 'MEANS', 'DETERMING', 'APPARATUS', 'PARTICULAR']
[(('JOURNAL', 'NOUN'), 0.7448), (('TRANSACTION', 'NOUN'), 0.7391), (('INTERNAL', 'NOUN'), 0.737), (('CIRCUIT', 'NOUN'), 0.737), (('SUCCESS', 'NOUN'), 0.7332), (('SECTION', 'NOUN'), 0.7303), (('PANEL', 'NOUN'), 0.7296), (('FALSE', 'ADJ'), 0.7291), (('FILES', 'NOUN'), 0.7289), (('2ND', 'NOUN'), 0.7281)]
[(('images', 'NOUN'), 0.7981), (('actual image', 'NOUN'), 0.7964), (('other image', 'NOUN'), 0.77), (('original image', 'NOUN'), 0.7657), (('full image', 'NOUN'), 0.757), (('just the image', 'NOUN'), 0.7519), (('source image', 'NOUN'), 0.7441), (('specific image', 'NOUN'), 0.7431), (('entire image', 'NOUN'), 0.7429), (('just an image', 'NOUN'), 0.7311)]
['MICROPROCESSOR', 'SUB-UNIT', 'unit', 'Multiprocessors', 'pre-processing', 'Sub-Units', 'MICROCONTROLLER', 'DEVICE', 'chip', 'MICROCHIP']
['ENVIROMENT', 'management', 'MANGEMENT', 'ENLARGE', 'image', 'ENVIRONMENTS', 'development', 'DEVELOPEME

[(('specific information', 'NOUN'), 0.8361), (('other information', 'NOUN'), 0.8307), (('detailed information', 'NOUN'), 0.8288), (('much information', 'NOUN'), 0.8234), (('pertinent information', 'NOUN'), 0.823), (('basic information', 'NOUN'), 0.815), (('such information', 'NOUN'), 0.8126), (('valuable information', 'NOUN'), 0.8124), (('important information', 'NOUN'), 0.81), ((' information', 'NOUN'), 0.7978)]
['Airlifter', 'MAINTENANCE', 'non-operational', 'information', 'Informaton', 'EQUIPEMENT', 'OPERATIONS', 'Necesary', 'SEVICES', 'services']
[(('assembly', 'ADV'), 0.8344), (('assemblies', 'NOUN'), 0.8071), (('tooling', 'NOUN'), 0.7097), (('assembler', 'NOUN'), 0.7066), (('Assembly', 'ORG'), 0.6887), (('disassembly', 'NOUN'), 0.6778), (('assemble', 'VERB'), 0.6703), (('basic code', 'NOUN'), 0.6557), (('instruction', 'NOUN'), 0.6557), (('critical sections', 'NOUN'), 0.6552)]
['RETRIEVED', '.', 'computer-readable', 'CONTENTS', 'storage', 'safe-keeping', 'data', 'DATASET', 'files'

[(('referencing', 'VERB'), 0.8334), (('refrence', 'NOUN'), 0.8184), (('only reference', 'NOUN'), 0.7852), (('just a reference', 'NOUN'), 0.7835), (('specific reference', 'NOUN'), 0.7649), (('original reference', 'NOUN'), 0.7614), (('direct reference', 'NOUN'), 0.7477), (('Referencing', 'VERB'), 0.7465), (('references', 'VERB'), 0.7448), (('refference', 'NOUN'), 0.7434)]
['system', 'SYTEM', 'REWIRED', 'systems', 'SYTEMS', 'Entire', 'entire', 'Non-Electrical', 'INSTALLATIONS', 'control']
['YET', 'yet', 'ANother', 'Another', 'another', 'One', 'OnE', 'one', 'but', 'But']
[(('datasheet', 'NOUN'), 0.7581), (('circuit diagram', 'NOUN'), 0.7097), (('pinout', 'NOUN'), 0.7019), (('diagram', 'NOUN'), 0.7017), (('wiring diagram', 'NOUN'), 0.6951), (('data sheet', 'NOUN'), 0.694), (('microcontroller', 'NOUN'), 0.686), (('motherboard manual', 'NOUN'), 0.6844), (('teensy', 'NOUN'), 0.6803), (('datasheets', 'NOUN'), 0.6762)]
[(('threshhold', 'NOUN'), 0.918), (('certain threshold', 'NOUN'), 0.8742), ((